In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import resnet50, ResNet50_Weights
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
data_transform = transforms.Compose([
        #transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                     std=[0.229, 0.224, 0.225])
    ])

mitosis_dataset = datasets.ImageFolder(root='../MITOS_Datasets/Data_CMC_COADEL_224_1/train',
                                           transform=data_transform)
dataset_loader = torch.utils.data.DataLoader(mitosis_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=4)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = '../MITOS_Datasets/Data_CMC_COADEL_224_1/'

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [5]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [7]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [8]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        #model_ft = models.resnet18(pretrained=use_pretrained)
        #model_ft = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        model_ft = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "resnet152":
        """ Resnet152
        """
        model_ft = models.resnet152(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 224    

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 128#224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [11]:
model_name = "resnet"
model_ft, input_size = initialize_model(model_name, 2, feature_extract, use_pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/jmwolf/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [12]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

Initializing Datasets and Dataloaders...


In [13]:
dataloaders_dict = {x: torch.utils.data.DataLoader(mitosis_dataset, batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

In [14]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [15]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 0.6331 Acc: 0.6914
val Loss: 0.5758 Acc: 0.7187

Epoch 1/14
----------
train Loss: 0.6370 Acc: 0.6922
val Loss: 0.5828 Acc: 0.7293

Epoch 2/14
----------
train Loss: 0.6260 Acc: 0.6948
val Loss: 0.5705 Acc: 0.7242

Epoch 3/14
----------
train Loss: 0.6309 Acc: 0.6931
val Loss: 0.5943 Acc: 0.7278

Epoch 4/14
----------
train Loss: 0.6240 Acc: 0.6951
val Loss: 0.5766 Acc: 0.7203

Epoch 5/14
----------
train Loss: 0.6327 Acc: 0.6925
val Loss: 0.5875 Acc: 0.7261

Epoch 6/14
----------
train Loss: 0.6260 Acc: 0.6943
val Loss: 0.5933 Acc: 0.6875

Epoch 7/14
----------
train Loss: 0.6321 Acc: 0.6919
val Loss: 0.5783 Acc: 0.6953

Epoch 8/14
----------
train Loss: 0.6252 Acc: 0.6951
val Loss: 0.6372 Acc: 0.7253

Epoch 9/14
----------
train Loss: 0.6273 Acc: 0.6959
val Loss: 0.5671 Acc: 0.7269

Epoch 10/14
----------
train Loss: 0.6317 Acc: 0.6940
val Loss: 0.5840 Acc: 0.7111

Epoch 11/14
----------
train Loss: 0.6303 Acc: 0.6955
val Loss: 0.5794 Acc: 0.6942

Ep

In [17]:
optimizer_ft = optim.Adam(params_to_update, lr=0.001)#, momentum=0.9)
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 0.5892 Acc: 0.7056
val Loss: 0.5621 Acc: 0.7125

Epoch 1/14
----------
train Loss: 0.5847 Acc: 0.7113
val Loss: 0.5807 Acc: 0.7299

Epoch 2/14
----------
train Loss: 0.5853 Acc: 0.7080
val Loss: 0.5903 Acc: 0.7280

Epoch 3/14
----------
train Loss: 0.5878 Acc: 0.7080
val Loss: 0.5541 Acc: 0.7292

Epoch 4/14
----------
train Loss: 0.5882 Acc: 0.7096
val Loss: 0.5808 Acc: 0.7264

Epoch 5/14
----------
train Loss: 0.5832 Acc: 0.7108
val Loss: 0.5497 Acc: 0.7293

Epoch 6/14
----------
train Loss: 0.5858 Acc: 0.7115
val Loss: 0.5503 Acc: 0.7242

Epoch 7/14
----------
train Loss: 0.5853 Acc: 0.7104
val Loss: 0.6103 Acc: 0.6671

Epoch 8/14
----------
train Loss: 0.5872 Acc: 0.7092
val Loss: 0.5836 Acc: 0.7281

Epoch 9/14
----------
train Loss: 0.5864 Acc: 0.7105
val Loss: 0.5646 Acc: 0.7099

Epoch 10/14
----------
train Loss: 0.5844 Acc: 0.7096
val Loss: 0.5721 Acc: 0.7287

Epoch 11/14
----------
train Loss: 0.5828 Acc: 0.7110
val Loss: 0.5561 Acc: 0.7164

Ep

In [18]:
torch.save(model_ft, 'resnet50_mitosis.pth')

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 0.5628 Acc: 0.7200
val Loss: 0.5528 Acc: 0.7272

Epoch 1/14
----------
train Loss: 0.5531 Acc: 0.7243
val Loss: 0.5463 Acc: 0.7312

Epoch 2/14
----------
train Loss: 0.5513 Acc: 0.7237
val Loss: 0.5421 Acc: 0.7316

Epoch 3/14
----------
train Loss: 0.5471 Acc: 0.7256
val Loss: 0.5526 Acc: 0.7328

Epoch 4/14
----------
train Loss: 0.5469 Acc: 0.7270
val Loss: 0.5452 Acc: 0.7342

Epoch 5/14
----------
train Loss: 0.5468 Acc: 0.7279
val Loss: 0.5628 Acc: 0.7296

Epoch 6/14
----------
train Loss: 0.5432 Acc: 0.7291
val Loss: 0.5596 Acc: 0.7337

Epoch 7/14
----------
train Loss: 0.5465 Acc: 0.7292
val Loss: 0.5693 Acc: 0.7291

Epoch 8/14
----------
train Loss: 0.5431 Acc: 0.7304
val Loss: 0.5475 Acc: 0.7352

Epoch 9/14
----------
train Loss: 0.5449 Acc: 0.7282
val Loss: 0.5465 Acc: 0.7351

Epoch 10/14
----------
train Loss: 0.5443 Acc: 0.7288
val Loss: 0.5594 Acc: 0.7356

Epoch 11/14
----------
train Loss: 0.5441 Acc: 0.7280
val Loss: 0.5728 Acc: 0.7289

Ep

In [ ]:
torch.save(model_ft, 'resnet50_v2_mitosis.pth')